In [22]:
import torch
import numpy as np
from scipy.ndimage import uniform_filter

In [23]:
year = "2020"
month = "02"
day = "09"
hour = "06"
minute = "15"

In [24]:
lead_time = 6

In [25]:
base_dir = f"/gws/nopw/j04/wiser_ewsa/mrakotomanga/Intercomparison/combined_nowcasts/t{lead_time}"

nowcast_origin = f"{year}{month}{day}_{hour}{minute}.pt"

file = f"{base_dir}/{nowcast_origin}"

In [26]:
nowcast = torch.load(file, weights_only=False)

In [27]:
gt = nowcast["ground_truth"]
zcast = nowcast["zcast"]
nflics = nowcast["nflics"]
netncc = nowcast["netncc"]

In [28]:
def compute_fss(pred, obs, window):
    """
    Compute Fractions Skill Score for a given spatial window size.
    Uses a uniform filter to calculate local fractions.
    """
    pred = np.clip(pred, 0, 1)
    obs = np.clip(obs, 0, 1)

    # Local fractional coverage (mean in the neighbourhood)
    f_pred = uniform_filter(pred, size=window, mode="constant")
    f_obs  = uniform_filter(obs, size=window, mode="constant")

    numerator = np.mean((f_pred - f_obs) ** 2)
    denominator = np.mean(f_pred ** 2 + f_obs ** 2)
    fss = 1 - numerator / (denominator + 1e-8)
    return fss

In [29]:
windows = [3, 9, 25, 49, 81, 121]
results = []
for w in windows:
    fss_zcast = compute_fss(zcast, gt, w)
    fss_nflics = compute_fss(nflics, gt, w)
    fss_netncc = compute_fss(netncc, gt, w)
    results.append({
        "window": w,
        "fss_zcast": fss_zcast,
        "fss_nflics": fss_nflics,
        "fss_netncc": fss_netncc
    })

# ---------------------------------------------------------------------
# Display results
# ---------------------------------------------------------------------
print(f"{'Window':>8} | {'ZCAST':>10} | {'NFLICS':>10} | {'NetNCC':>10}")
print("-" * 46)
for r in results:
    print(f"{r['window']:>8} | {r['fss_zcast']:.4f} | {r['fss_nflics']:.4f} | {r['fss_netncc']:.4f}")

  Window |      ZCAST |     NFLICS |     NetNCC
----------------------------------------------
       3 | 0.1541 | 0.0580 | 0.0817
       9 | 0.2058 | 0.0589 | 0.0871
      25 | 0.3465 | 0.0600 | 0.0904
      49 | 0.4880 | 0.0600 | 0.0911
      81 | 0.5844 | 0.0602 | 0.0953
     121 | 0.6411 | 0.0606 | 0.0990
